# Imports

In [18]:
# Imports
import numpy as np
import random

# Introduction

## Column Vectors

In [19]:
x = np.array([[0.75], [0.25]])
print(x.sum())

1.0


## Problem 1: Generating Random Boolean Functions
### Problem Description
The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations). Write a Python function random_constant_balanced that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

### Defining four constant and balanced functions

In [20]:
# Const func
def f1(a, b, c, d):
    "Always returns 0"
    return 0

# balanced function
def f2(a, b, c, d):
    "Identity function"
    return a

# balanced function
def f3(a, b, c, d):
    "Not function"
    return a ^ b ^  c ^ d

# const function
def f4(a, b, c, d):
    "const function"
    return 1

def random_tuple(n):
  """Returns a random constant or balanced tuple of length 2**n."""
  # Select a type of function - 50/50 choice between constant and balanced.
  ftype = random.choice(['constant', 'balanced'])
  # If constant is randomly selected.
  if ftype == 'constant':
    # Choose whether it's all 0's or all 1's. - again a 50/50 choice.
    zero_or_one = random.choice([0, 1])
    # Create tuples of length n and return.
    # Note (x,) is a tuple of length one containing x.
    # Mutliplying a tuple (or list) by an int i gives a new list with the 
    # elements from the original list repeated i times.
    # e.g. (1, ) * 3 = (1, 1, 1)
    t = (zero_or_one,) * 2**n
  # Otherwise balanced.
  else:
    # List with half 0's, half 1's (in that order).
    # We use a list here because random.shuffle works inplace.
    t = ([0,] * 2**(n-1)) + ([1,] * 2**(n-1))
    # Shuffle bits so that the 1's are randomly placed in the tuple.
    random.shuffle(t)
    # Make it a tuple.
    t = tuple(t)
  return t

def bin_args_to_int(*args):
  """Take an arbitary number of binary args and convert to an int."""
  # Accept any truthy values for a '1' and likewise for a '0'.
  bits = ['1' if i else '0' for i in args]
  # Join the bits into a single string.
  bits = ''.join(bits)
  # Convert the binary string to an int.
  bits_int = int(bits, 2)
  # Convert to an int and return. The 2 tells 
  return bits_int

# fclosure
def random_constant_balanced():
  """Return a function that uses an object in the enclosing function."""
  # Use our random_tuple function form above to get a constant of balanced
  # tuple of length 2**n.
  f_a = random_tuple(4)
  # Define a function that takes n binary arguments, converts them to an int
  # using our bin_args_to_int function and then selects that element from f_a.
  def f(*args):
    if len(args) != 4:
      return None
    else:
      return f_a[bin_args_to_int(*args)]
  # Return the closure.
  return f

random_function = random_constant_balanced()
print(random_function(1, 0, 0, 0))

1


## Problem 2: Classical Testing for Function Type

## Problem 3: Quantum Oracles

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm